# Capstone Project: [testfit.io](https://blog.testfit.io/)
## Wrangle

In [2]:
import numpy as np
import pandas as pd

import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import wrangle as wr


Bad key "text.kerning_factor" on line 4 in
/usr/local/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [ ]:
pd.set_option("display.max_columns", None)
plt.rc("figure", figsize=(16, 8))

## Stakeholder Notes
- Work with Commitments
- Activity Descriptions == New Construction, SubRehab, Additions, and Improvements
- Loan amount >= $1M:
    - Mortgage at time of Firm Commitment Issuance, Amendment or Reissuance?
    - Mortgage at Firm Commitment Issuance?
    - Final Mortgage Amount? **Probably this one...**
- Market == Large urban centers; meaning **cities** not just states; not rural one-off projects
- Types of FHA loan is irrelevant

In [ ]:
# commitments
# df  = pd.read_excel("Initi_Endores_Firm Comm_DB_FY06_FY20_Q2.xlsx", sheet_name="Firm Cmtmts, Iss'd and Reiss'd", header=6)
# print(df.shape)
# df.head()
df = pd.read_csv("hud_commitments.csv", index_col=0)
print(df.shape)
df.head()

In [ ]:
# df["Activity Description"].value_counts()

In [ ]:
# df.to_csv("hud_commitments.csv")

In [ ]:
# df.rename(columns={"Activity Group": "activity_group"}, inplace=True)

In [ ]:
# activity_groups filtered for groups of interest for clifton
# df = df[(df.activity_group == "NC/SR") | (df.activity_group == "Imp/Adds")]

In [ ]:
# only including loans greater than or equal to $1M
# df = df[df["Final Mortgage Amount"] >= 1000000]
# df.head()

In [ ]:
df.info()

In [ ]:
# df["Activity Description"].value_counts()

In [ ]:
# df["activity_group"].value_counts()

In [ ]:
df.rename(columns={"Facility Type": "facility_type"}, inplace=True)

In [ ]:
df.facility_type.value_counts()

In [ ]:
df.rename(columns={"FHA Number": "fha_number"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).fha_number.count().sort_values().plot.barh()
plt.show()

In [ ]:
# df[df["Final Mortgage Amount"] >= 1000000].shape

In [ ]:
df.rename(columns={"Final Mortgage Amount": "final_mortgage"}, inplace=True)

In [ ]:
df.groupby(["Project State"]).final_mortgage.sum().sort_values().plot.barh()

In [ ]:
df["Project City"].value_counts().head(20)

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"]).size().sort_values()

In [ ]:
df.groupby(["Project City", "Fiscal Year of Firm Commitment"])["final_mortgage"].sum().sort_values()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
df[df["Project City"] == "Minneapolis"].groupby("Fiscal Year of Firm Commitment")["final_mortgage"].sum()

In [ ]:
# csv works and can be read
df = pd.read_csv("hud_commitments.csv", index_col=0)
df.head()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].sum()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].count()

In [ ]:
df[df["Project City"] == "Austin"].groupby("Fiscal Year of Firm Commitment")["Final Mortgage Amount"].size()

In [ ]:
df.shape

In [ ]:
# let's see the number of unique values for earch column
# for col in df.columns:
#     unique = len(df[col].unique())
#     if unique > 10:
#         print()
#         print(f'{col} has {unique} values')
#     else:
#         print()
#         print(f'{col} value count:')
#         print(df[col].value_counts())

There are hospitals in this data.

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
# df["Date of Firm Issue"] = pd.to_datetime(df["Date of Firm Issue"])
# df["Date of Firm Issue"].dt.year

---
## Wrangle

In [ ]:
df = wr.wrangle_hud()
print(df.shape)
df.head()

In [ ]:
df.info()

---
## Explore
### What are the characteristics of loan practices in Houston (2009), Seattle (2010), and Dallas (2012)?
- Do these markets follow a similar trajectory in the years leading up the boom?

In [ ]:
# fmt-clipboard function from terminal == black formatting
# df[df.project_city == "Houston"].groupby(["date_of_firm_commitment_activity"])[
#     "final_mortgage_amount"
# ].sum().reset_index().sort_values(by="date_of_firm_commitment_activity")

In [ ]:
# clifton's time machine markets
threshold_cities = ["Houston", "Seattle", "Dallas"]

# loop through 
for city in threshold_cities:
    # subset for city
    date_index_df = df[df.project_city == city].sort_values(
        "date_of_firm_commitment_activity"
    ).set_index("date_of_firm_commitment_activity")

    # sum of the final_mortgage_amount by date_of_firm_commitment_activity
    date_index_df = (
        date_index_df.groupby(["date_of_firm_commitment_activity"])[
            "final_mortgage_amount"
        ]
        .sum()
        .reset_index()
    )
    
    date_index_df.set_index("date_of_firm_commitment_activity", inplace=True)
    
    # plot
    date_index_df.plot()  
    plt.title(f"Final Mortgage Amount Over Time: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()
    
    date_index_df.resample('Y').mean().plot()
    plt.title(f"Yearly Average Final Mortgage Amount: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()
    
    date_index_df.resample('6M').mean().plot()
    plt.title(f"6 Month Average Final Mortgage Amount: {city}")
    plt.xlabel("Date of Firm Commitment Activity")
    plt.ylabel("Final Mortgage Amount")
    plt.show()

**Takeaway**: What I observe in the charts above is that for the years that Clifton would have liked to enter into a market, the total final mortgage production for that city droped to $1M or less.